# Accompanying notebook to Optimization Stories in Logistics and Transportation

By Joaquim Gromicho, ORTEC & The Analytics Academy, 2020

## Dependencies

In [1]:
import pyomo.environ as pyo
import fractions
import numpy as np
import matplotlib.pyplot as plt

## Caroline's Trophy Factory

Caroline owns a company that produces trophies for
* football
 * wood base, engraved plaque, brass football on top
 * €12 profit and uses 4 dm of wood
* golf
 * wood base, engraved plaque, golf ball on top
 * €9 profit and uses 2 dm of wood

Caroline’s current stock of raw materials
* 1000 footballs
* 1500 golf balls
* 1750 plaques
* 480 m (4800 dm) of wood

> Caroline wonders what the optimal production plan should be, in other words: how many football and how many golf trophies should Betty produce to maximize his profit while respecting the availability of raw materials?

***

The following model __maximizes__ Caroline's profit by deciding the number of $x_1$ football and $x_2$ golf trophies to produce.

$$
\begin{array}{rrcrcl}
\max    & 12x_1 & + & 9x_2               \\
s.t.    &   x_1 &   &      & \leq & 1000 \\
        &       &   &  x_2 & \leq & 1500 \\
        &   x_1 & + &  x_2 & \leq & 1750 \\
        &  4x_1 & + & 2x_2 & \leq & 4800 \\
        &   x_1 & , &  x_2 & \geq & 0    \\
\end{array}
$$


In [2]:
Caroline = pyo.ConcreteModel('Caroline')

Caroline.football = pyo.Var(domain=pyo.NonNegativeReals)
Caroline.golf     = pyo.Var(domain=pyo.NonNegativeReals)

Caroline.profit    = pyo.Objective(expr = 12*Caroline.football + 9*Caroline.golf, sense=pyo.maximize)

Caroline.footballs = pyo.Constraint(expr =   Caroline.football                   <= 1000)
Caroline.golfBalls = pyo.Constraint(expr =                         Caroline.golf <= 1500)
Caroline.plaques   = pyo.Constraint(expr =   Caroline.football +   Caroline.golf <= 1750)
Caroline.wood      = pyo.Constraint(expr = 4*Caroline.football + 2*Caroline.golf <= 4800)
 
from pyomo.opt import SolverFactory
%time results = SolverFactory('ipopt').solve(Caroline)

print(results.solver.status, results.solver.termination_condition )
print(Caroline.profit.expr())

Caroline.display()

Wall time: 270 ms
ok optimal
17700.00017699499
Model Caroline

  Variables:
    football : Size=1, Index=None
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :     0 : 650.0000064995821 :  None : False : False : NonNegativeReals
    golf : Size=1, Index=None
        Key  : Lower : Value              : Upper : Fixed : Stale : Domain
        None :     0 : 1100.0000110000003 :  None : False : False : NonNegativeReals

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 17700.00017699499

  Constraints:
    footballs : Size=1
        Key  : Lower : Body              : Upper
        None :  None : 650.0000064995821 : 1000.0
    golfBalls : Size=1
        Key  : Lower : Body               : Upper
        None :  None : 1100.0000110000003 : 1500.0
    plaques : Size=1
        Key  : Lower : Body               : Upper
        None :  None : 1750.0000174995826 : 1750.0
    wood : Size=1
        

In [3]:
from pyomo.environ import SolverManagerFactory
%time results = SolverManagerFactory('neos').solve( Caroline, opt='cplex')

Wall time: 5min 4s


In [4]:
print(results.solver.status, results.solver.termination_condition )
print(Caroline.profit.expr())

Caroline.display()

ok optimal
17700.0
Model Caroline

  Variables:
    football : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 650.0 :  None : False : False : NonNegativeReals
    golf : Size=1, Index=None
        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
        None :     0 : 1100.0 :  None : False : False : NonNegativeReals

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 17700.0

  Constraints:
    footballs : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 650.0 : 1000.0
    golfBalls : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 1100.0 : 1500.0
    plaques : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 1750.0 : 1750.0
    wood : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 4800.0 : 4800.0


In [5]:
%time results = SolverFactory('cbc',executable='C:/Users/joaquimg/AppData/Local/Continuum/anaconda3/Lib/site-packages/pulp/solverdir/cbc/win/64/cbc.exe').solve(Caroline)

print(results.solver.status, results.solver.termination_condition )
print(Caroline.profit.expr())

Caroline.display()

Wall time: 260 ms
ok optimal
17700.0
Model Caroline

  Variables:
    football : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 650.0 :  None : False : False : NonNegativeReals
    golf : Size=1, Index=None
        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
        None :     0 : 1100.0 :  None : False : False : NonNegativeReals

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 17700.0

  Constraints:
    footballs : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 650.0 : 1000.0
    golfBalls : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 1100.0 : 1500.0
    plaques : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 1750.0 : 1750.0
    wood : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 4800.0 : 4800.0


In [6]:
%time results = SolverFactory('glpk').solve(Caroline)

print(results.solver.status, results.solver.termination_condition )
print(Caroline.profit.expr())

Caroline.display()

Wall time: 367 ms
ok optimal
17700.0
Model Caroline

  Variables:
    football : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 650.0 :  None : False : False : NonNegativeReals
    golf : Size=1, Index=None
        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
        None :     0 : 1100.0 :  None : False : False : NonNegativeReals

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 17700.0

  Constraints:
    footballs : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 650.0 : 1000.0
    golfBalls : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 1100.0 : 1500.0
    plaques : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 1750.0 : 1750.0
    wood : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 4800.0 : 4800.0


In [8]:
%time results = SolverFactory('cplex').solve(Caroline)

print(results.solver.status, results.solver.termination_condition )
print(Caroline.profit.expr())

Caroline.display()

Wall time: 247 ms
ok optimal
17700.0
Model Caroline

  Variables:
    football : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 650.0 :  None : False : False : NonNegativeReals
    golf : Size=1, Index=None
        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
        None :     0 : 1100.0 :  None : False : False : NonNegativeReals

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 17700.0

  Constraints:
    footballs : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 650.0 : 1000.0
    golfBalls : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 1100.0 : 1500.0
    plaques : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 1750.0 : 1750.0
    wood : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 4800.0 : 4800.0


In [9]:
%time results = SolverFactory('gurobi').solve(Caroline)

print(results.solver.status, results.solver.termination_condition )
print(Caroline.profit.expr())

Caroline.display()

Wall time: 428 ms
ok optimal
17700.0
Model Caroline

  Variables:
    football : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 650.0 :  None : False : False : NonNegativeReals
    golf : Size=1, Index=None
        Key  : Lower : Value  : Upper : Fixed : Stale : Domain
        None :     0 : 1100.0 :  None : False : False : NonNegativeReals

  Objectives:
    profit : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 17700.0

  Constraints:
    footballs : Size=1
        Key  : Lower : Body  : Upper
        None :  None : 650.0 : 1000.0
    golfBalls : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 1100.0 : 1500.0
    plaques : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 1750.0 : 1750.0
    wood : Size=1
        Key  : Lower : Body   : Upper
        None :  None : 4800.0 : 4800.0


## Accessing parts of the model

In [10]:
print( Caroline.football.value, Caroline.golf.value, Caroline.profit.expr() )

650.0 1100.0 17700.0


We can also query the model 

In [11]:
for v in Caroline.component_objects(pyo.Var, active=True):
    print ("Variable  ",v)
    varobject = getattr(Caroline, str(v))
    for index in varobject:
        print ("     ",index, varobject[index].value)
for o in Caroline.component_objects(pyo.Objective, active=True):
    print ("Objective ",o)
    varobject = getattr(Caroline, str(o))
    for index in varobject:
        print ("    ",index, varobject[index].expr())
for c in Caroline.component_objects(pyo.Constraint, active=True):
    print ("Constraint",c)
    varobject = getattr(Caroline, str(c))
    for index in varobject:
        print ("     ",index, varobject[index].uslack())

Variable   football
      None 650.0
Variable   golf
      None 1100.0
Objective  profit
     None 17700.0
Constraint footballs
      None 350.0
Constraint golfBalls
      None 400.0
Constraint plaques
      None 0.0
Constraint wood
      None 0.0


## Duals

Remember how to state the  __dual problem__ ?
    
$$
\begin{array}{rrcrcrcrcr}
\min    & 1000y_1 & + & 1500y_2 & + & 1750y_3 & + & 4800y_4 \\
s.t.    &     y_1 &   &         & + &     y_3 & + &    4y_4 & \geq & 12 \\
        &         &   &     y_2 & + &     y_3 & + &    2y_4 & \geq & 9 \\
        &     y_1 & , &     y_2 & + &     y_3 & , &     y_4 & \geq & 0  \\
\end{array}
$$

You may state this problem and solve it, but solving the primal also solves the dual and we may just ask the solution.

In [12]:
Caroline.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

%time results = SolverFactory('cbc', executable='C:/Users/joaquimg/AppData/Local/Continuum/anaconda3/Lib/site-packages/pulp/solverdir/cbc/win/64/cbc.exe').solve(Caroline)

# display all duals
print ("Duals")
for c in Caroline.component_objects(pyo.Constraint, active=True):
    print ("Constraint ",c)
    for index in c:
        print ("      ", index, str(fractions.Fraction(Caroline.dual[c[index]])))

Wall time: 226 ms
Duals
Constraint  footballs
       None 0
Constraint  golfBalls
       None 0
Constraint  plaques
       None -6
Constraint  wood
       None -3/2
